<a href="https://colab.research.google.com/github/ChoeTaeBin/Machine-Learnig/blob/main/airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##목적에 맞는 모델을 만드는 데 집중해 보았습니다.  
모델의 목표 : 사용자가 airbnb에 자신의 집을 올리려고 할때 얼마에 올려야 할지 제시해주는 시스템  
(우선 이렇게 가정하였습니다.)

In [ ]:
import pandas as pd
import numpy as np

##데이터 관찰

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Airbnb_Data.csv')
df.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,...,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,...,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,...,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,...,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,...,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0


In [ ]:
df.shape

(74111, 29)

샘플수: 74111  
특성수: 29

In [ ]:
features = df.columns
print('<특성들>\n', features)

<특성들>
 Index(['id', 'log_price', 'property_type', 'room_type', 'amenities',
       'accommodates', 'bathrooms', 'bed_type', 'cancellation_policy',
       'cleaning_fee', 'city', 'description', 'first_review',
       'host_has_profile_pic', 'host_identity_verified', 'host_response_rate',
       'host_since', 'instant_bookable', 'last_review', 'latitude',
       'longitude', 'name', 'neighbourhood', 'number_of_reviews',
       'review_scores_rating', 'thumbnail_url', 'zipcode', 'bedrooms', 'beds'],
      dtype='object')


각 특성을 조금 더 자세히 살펴 보겠습니다.

In [ ]:
for f in features:
  print(f'\n\n=======================< {f} >=========================\n')
  print(df[f].value_counts())



=======================< id >=========================

6901257     1
2460250     1
289037      1
16492690    1
14838028    1
           ..
14222061    1
6238463     1
15285110    1
11226321    1
3534845     1
Name: id, Length: 74111, dtype: int64


=======================< log_price >=========================

5.010635    2729
4.605170    2707
4.317488    2049
5.298317    1923
3.912023    1920
            ... 
6.999422       1
6.725034       1
7.070724       1
6.156979       1
6.357842       1
Name: log_price, Length: 767, dtype: int64


=======================< property_type >=========================

Apartment             49003
House                 16511
Condominium            2658
Townhouse              1692
Loft                   1244
Other                   607
Guesthouse              498
Bed & Breakfast         462
Bungalow                366
Villa                   179
Dorm                    142
Guest suite             123
Camper/RV                94
Timeshare             

##사용할 특성들  
(제가 주관적으로 판단한 부분이 많이 있습니다.)  
  

-> **target label: log_price**  
(가격에 ln을 씌운값, 달러로 추정됨)  
  

-> property_type  
상위 5개에 해당하는 샘플만 취하고 나머지는 버리기로 했습니다.(one-hot encoding후 특성수가 너무 많아지는 것을 방지하기 위함)   
Apartment             49003  
House                 16511  
Condominium(콘도)      2658  
Townhouse              1692  
Loft(다락방)           1244  
  

-> room_type  
Entire home/apt : 1  
Private room    : 2  
Shared room : 3  

으로 정수화 하겠습니다.  
  
->accomodates  
그대로 사용하겠습니다.  

->bathrooms  
가격과 많은 상관 관계가 있을 특성으로 생각하여 일단은 사용하겠습니다.  
그런데 욕실이 1.5개라는 것은 어떤의미인지 고민 해 봐야 할 것 같습니다.  
  
->bed_type  
Real Bed : 1  
etc : 0  
으로 하겠습니다.  
  
->cancellation_policy  
super_strict_30, super_strict_60 : 1  
strict: 2   
moderate : 3                  
flexible : 4    
로 정수화 하겠습니다.
  
->city  
위치에 관한 여러가지 정보들이 있는데, 이 중 city를 one-hot encoding해서 사용하겠습니다.  

->cleaning_fee, host_identity_verified, instant_bookable     
True : 1  
False : 0  
으로 정수화 하겠습니다.  
  
->beds  
그대로 사용하겠습니다.


##데이터 정리

In [ ]:
#property type이 가장 빈번한 5개에 해당하는 샘플만 살림
refined_df = df[df['property_type'].isin(['Apartment', 'House', 'Condominium', 'Townhouse', 'Loft'])]

In [ ]:
refined_df['property_type'].value_counts()

Apartment      49003
House          16511
Condominium     2658
Townhouse       1692
Loft            1244
Name: property_type, dtype: int64

In [ ]:
refined_df.shape

(71108, 29)

In [ ]:
#일단 사용할 특징들만 남기자

refined_df = refined_df[['log_price', 'property_type', 'accommodates', 'bathrooms',
                          'bed_type', 'cancellation_policy', 'city', 'cleaning_fee',
                          'host_identity_verified', 'instant_bookable', 'beds']]

In [ ]:
refined_df.shape

(71108, 11)

##NaN값 처리

In [ ]:
refined_df.isnull().sum()

log_price                   0
property_type               0
accommodates                0
bathrooms                 181
bed_type                    0
cancellation_policy         0
city                        0
cleaning_fee                0
host_identity_verified    185
instant_bookable            0
beds                      123
dtype: int64

NaN값을 가진 sample이 그리 많지 않으므로 그냥 삭제하기로 했다.

In [ ]:
print('NaN값 삭제전: ',refined_df.shape[0])
refined_df.dropna(inplace = True)
print('NaN값 삭제후: ',refined_df.shape[0])

NaN값 삭제전:  71108
NaN값 삭제후:  70662


In [ ]:
#True, False, 't', 'f' -> 0,1로
refined_df = refined_df.replace({True: 1, 't' : 1,  False : 0, 'f': 0})

In [ ]:
refined_df

,log_price,property_type,accommodates,bathrooms,bed_type,cancellation_policy,city,cleaning_fee,host_identity_verified,instant_bookable,beds
0,5.010635,Apartment,3,1.0,Real Bed,strict,NYC,1,1,0,1.0
1,5.129899,Apartment,7,1.0,Real Bed,strict,NYC,1,0,1,3.0
2,4.976734,Apartment,5,1.0,Real Bed,moderate,NYC,1,1,1,3.0
3,6.620073,House,4,1.0,Real Bed,flexible,SF,1,1,0,2.0
4,4.744932,Apartment,2,1.0,Real Bed,moderate,DC,1,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
74105,4.248495,House,2,1.0,Real Bed,moderate,LA,1,1,0,1.0
74106,4.605170,Apartment,1,1.0,Real Bed,flexible,NYC,0,1,0,1.0
74107,5.043425,Apartment,4,2.0,Real Bed,moderate,LA,1,0,0,4.0
74108,5.220356,Apartment,5,1.0,Real Bed,moderate,NYC,1,1,1,2.0


In [ ]:
#bed_type정수화
refined_df['bed_type'] = refined_df['bed_type'].replace({'Real Bed': 1})#Real Bed는 1
refined_df['bed_type'] = refined_df['bed_type'].where(refined_df['bed_type'] == 1, other=0)#나머지는 0

In [ ]:
refined_df['bed_type'].value_counts()

1    68700
0     1962
Name: bed_type, dtype: int64

In [ ]:
#cancelation policy 정수화
refined_df['cancellation_policy'] = refined_df['cancellation_policy'].replace({
    'super_strict_30': 1,
    'super_strict_60' : 1,
    'strict' : 2,
    'moderate' : 3,
    'flexible' : 4})

In [ ]:
refined_df['cancellation_policy'].value_counts()

2    30941
4    21475
3    18124
1      122
Name: cancellation_policy, dtype: int64

In [ ]:
refined_df

,log_price,property_type,accommodates,bathrooms,bed_type,cancellation_policy,city,cleaning_fee,host_identity_verified,instant_bookable,beds
0,5.010635,Apartment,3,1.0,1,2,NYC,1,1,0,1.0
1,5.129899,Apartment,7,1.0,1,2,NYC,1,0,1,3.0
2,4.976734,Apartment,5,1.0,1,3,NYC,1,1,1,3.0
3,6.620073,House,4,1.0,1,4,SF,1,1,0,2.0
4,4.744932,Apartment,2,1.0,1,3,DC,1,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
74105,4.248495,House,2,1.0,1,3,LA,1,1,0,1.0
74106,4.605170,Apartment,1,1.0,1,4,NYC,0,1,0,1.0
74107,5.043425,Apartment,4,2.0,1,3,LA,1,0,0,4.0
74108,5.220356,Apartment,5,1.0,1,3,NYC,1,1,1,2.0


##one-hot encoding

In [ ]:
encoded_col = pd.get_dummies(refined_df[['property_type', 'city']])

In [ ]:
encoded_col

,property_type_Apartment,property_type_Condominium,property_type_House,property_type_Loft,property_type_Townhouse,city_Boston,city_Chicago,city_DC,city_LA,city_NYC,city_SF
0,1,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
74105,0,0,1,0,0,0,0,0,1,0,0
74106,1,0,0,0,0,0,0,0,0,1,0
74107,1,0,0,0,0,0,0,0,1,0,0
74108,1,0,0,0,0,0,0,0,0,1,0


In [ ]:
refined_df = pd.concat([refined_df, encoded_col], axis = 1)

In [ ]:
refined_df

,log_price,property_type,accommodates,bathrooms,bed_type,cancellation_policy,city,cleaning_fee,host_identity_verified,instant_bookable,...,property_type_Condominium,property_type_House,property_type_Loft,property_type_Townhouse,city_Boston,city_Chicago,city_DC,city_LA,city_NYC,city_SF
0,5.010635,Apartment,3,1.0,1,2,NYC,1,1,0,...,0,0,0,0,0,0,0,0,1,0
1,5.129899,Apartment,7,1.0,1,2,NYC,1,0,1,...,0,0,0,0,0,0,0,0,1,0
2,4.976734,Apartment,5,1.0,1,3,NYC,1,1,1,...,0,0,0,0,0,0,0,0,1,0
3,6.620073,House,4,1.0,1,4,SF,1,1,0,...,0,1,0,0,0,0,0,0,0,1
4,4.744932,Apartment,2,1.0,1,3,DC,1,1,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74105,4.248495,House,2,1.0,1,3,LA,1,1,0,...,0,1,0,0,0,0,0,1,0,0
74106,4.605170,Apartment,1,1.0,1,4,NYC,0,1,0,...,0,0,0,0,0,0,0,0,1,0
74107,5.043425,Apartment,4,2.0,1,3,LA,1,0,0,...,0,0,0,0,0,0,0,1,0,0
74108,5.220356,Apartment,5,1.0,1,3,NYC,1,1,1,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
refined_df.drop(columns=['property_type','city'], inplace = True) #property_type, city 삭제

In [ ]:
refined_df.columns

Index(['log_price', 'accommodates', 'bathrooms', 'bed_type',
       'cancellation_policy', 'cleaning_fee', 'host_identity_verified',
       'instant_bookable', 'beds', 'property_type_Apartment',
       'property_type_Condominium', 'property_type_House',
       'property_type_Loft', 'property_type_Townhouse', 'city_Boston',
       'city_Chicago', 'city_DC', 'city_LA', 'city_NYC', 'city_SF'],
      dtype='object')

##테스트 세트 분리

In [ ]:
from sklearn.model_selection import train_test_split

train_input,test_input, train_target, test_target = train_test_split(refined_df.iloc[:,1 : ], refined_df['log_price'], random_state = 42) #테스트 세트 분리

In [ ]:
train_input.shape, test_target.shape

((52996, 19), (17666,))

##표준화

In [45]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
std_scaler.fit(train_input)
train_input = std_scaler.transform(train_input)
test_input = std_scaler.transform(test_input)

(17666,)

##모델 생성, 훈련

In [44]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# SGDRegressor 모델 초기화
sgd_reg = SGDRegressor()

# 교차 검증을 통한 모델 평가
rmse_scores = cross_val_score(sgd_reg, train_input, train_target, cv=5, scoring='neg_root_mean_squared_error')
r2_scores = cross_val_score(sgd_reg, train_input, train_target, cv=5, scoring='r2')

# 테스트 데이터에 대한 모델 평가
sgd_reg.fit(train_input, train_target)
test_pred = sgd_reg.predict(test_input)
test_rmse = np.sqrt(mean_squared_error(test_target, test_pred))
test_r2 = r2_score(test_target, test_pred)

print("Cross-Validation RMSE Scores:", -rmse_scores)
print("Cross-Validation R2 Scores:", r2_scores)

print("Average RMSE:", -np.mean(rmse_scores))
print("Average R2 Score:", np.mean(r2_scores))

print("Test RMSE:", test_rmse)
print("Test R2 Score:", test_r2)


Cross-Validation RMSE Scores: [0.55397836 0.55636657 0.56490189 0.54614648 0.55261968]
Cross-Validation R2 Scores: [0.40135903 0.40365966 0.38509136 0.40266597 0.40006954]
Average RMSE: 0.5548025958368183
Average R2 Score: 0.39856911138726486
Test RMSE: 1.487542545007487
Test R2 Score: -3.3942226060847505


점수가 잘안나온다. 특히, test 점수가 잘안나온다. 모델이 아직 데이터를 학습하지 못하고 있다고 판단된다. 성능을 높이기위해서는 동네당 평균 집값정보를 찾아서 추가하면 좋을 것 같다.